# Récupération jeu de données

L'objectif de ce notebook

Pour la documentation api Spotify



*   https://spotifypy.readthedocs.io/en/latest/api.html
*   https://github.com/spotipy-dev/spotipy/blob/master/examples/search.py
*   https://pypi.org/project/spotipy/








## Installation de l'api SPOTIFY

In [1]:
 # Installation des packages pour Spotify
#pip install spotipy 

## Import des librairies

In [2]:
from function.utils import tqdm, glob, json, os
from function.utils import requests, BeautifulSoup #TODO : ligne a supprimer.

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Récupération des tokens

/!\ Attention /!\  
Pour pouvoir récuperer les access tokens de spotify il est impératif d'avoir un compte free ou premium.  
De plus avoir créer son APP.  
Par la suite, vous obtiendrez les client_id et client_id_secret permettant l'accès à l'API et le bon fonctionnement du code.

In [3]:
from function.function import get_token

PATH_TOKEN = "../data/token/*"
PATH_JSON = "../data/json_file/"
PATH_TXT = "../data/txt/"

liste_txt_file = glob.glob(PATH_TOKEN)

GENIUS_ACCESS_TOKEN = None
SPOTIFY_client_id = None
SPOTIFY_client_secret = None

GENIUS_ACCESS_TOKEN, SPOTIFY_client_id, SPOTIFY_client_secret = get_token(liste_txt_file)


In [4]:
from function.dictionnary import get_dictionnary

dictionnaire_song = get_dictionnary(PATH_JSON, "dictionnaire_song3.json")
liste_artiste = get_dictionnary(PATH_JSON, "dictionnaire_artiste.json")
data_artiste_titre = get_dictionnary(PATH_JSON, "data_artiste_titre.json")

#identifiant spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_client_id,
                                                           client_secret=SPOTIFY_client_secret))

### Pour un artiste dans la liste on met à jour les titres qu'il à réalisé.

In [5]:
set_artistes_existants = {artiste[1] for artiste in data_artiste_titre}

for nom_artiste in tqdm.tqdm(liste_artiste):
    if nom_artiste not in set_artistes_existants:
        results = sp.search(q=nom_artiste, limit=50) # requete de la recherche
        if 'tracks' in results and 'items' in results['tracks']:
            Info_titre = [(len(data_artiste_titre) + idx, nom_artiste, data['name']) for idx, data in enumerate(results['tracks']['items']) ]
            data_artiste_titre.extend(Info_titre)
        set_artistes_existants.add(nom_artiste)

             
with open(PATH_JSON+"data_artiste_titre.json", "w", encoding="utf-8") as json_file:
    json.dump(data_artiste_titre, json_file, indent=4, ensure_ascii=False)


100%|██████████| 3486/3486 [00:00<00:00, 1741880.36it/s]


In [6]:
from function.dictionnary import update_dictionary

# headers = {"User-Agent": "Mozilla/5.0"}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}


if os.path.exists("checkpoint.txt"):
    with open("checkpoint.txt", "r") as f:
        start_index = int(f.read().strip())
else:
    start_index = 0

save_every = 1000

In [ ]:
#TODO : Dans le scrapping il récupère aussi des livres     
for idx in tqdm.tqdm(range(start_index, len(data_artiste_titre))):
    artist_name, song_title = data_artiste_titre[idx][1:]
    try:
        if idx % save_every == 0:
            with open("checkpoint.txt", "w") as w:
                w.write(str(idx))

        # Si l'artiste est dans le dictionnaire et que le titre est présent
        if artist_name in dictionnaire_song and song_title in dictionnaire_song[artist_name].keys():

            # Si les paroles sont None ou vide tu mets à jour
            if dictionnaire_song[artist_name][song_title]["lyrics_primaire"] == None or dictionnaire_song[artist_name][song_title]["lyrics_primaire"] == "":
                dictionnaire_song = update_dictionary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)
            # Sinon tu passes
            continue
        else:
            dictionnaire_song = update_dictionary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)
    except KeyboardInterrupt:
        print(f"\nInterruption manuelle à l'index {idx}. Sauvegarde en cours...")
        with open("checkpoint.txt", "w") as w:
            w.write(str(idx))
        with open(PATH_JSON+"dictionnaire_song3.json", "w", encoding="UTF-8") as json_file:
            json.dump(dictionnaire_song, json_file, indent=4)    
        break
    except Exception as e:
        print(f"erreur à l'index {idx} : {e}")

        with open("checkpoint.txt", "w") as w:
            w.write(str(idx))
        break


  1%|          | 1314/173289 [1:00:53<132:49:00,  2.78s/it]


KeyboardInterrupt: 

In [8]:
with open(PATH_JSON+"dictionnaire_song3.json", "w", encoding="UTF-8") as json_file:
    json.dump(dictionnaire_song, json_file, indent=4)